In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
pd.set_option('display.max_columns', None)

%matplotlib inline

In [8]:
df = pd.read_csv('./student_data.csv')

### Exploratory Data Analysis (EDA)

In [9]:
df.head()

,School,Gender,Age,Address,Family_Size,Parental_Status,Mother_Education,Father_Education,Mother_Job,Father_Job,Reason_for_Choosing_School,Guardian,Travel_Time,Study_Time,Number_of_Failures,School_Support,Family_Support,Extra_Paid_Class,Extra_Curricular_Activities,Attended_Nursery,Wants_Higher_Education,Internet_Access,In_Relationship,Family_Relationship,Free_Time,Going_Out,Weekend_Alcohol_Consumption,Weekday_Alcohol_Consumption,Health_Status,Number_of_Absences,Grade_1,Grade_2,Final_Grade,Dropped_Out
0,School A,Female,13.0,Urban,1.0,Together,Primary,Secondary,Business,Healthcare,Reputation,Mother,14.0,5.0,0,No,Yes,Yes,No,No,Yes,No,Yes,Bad,5.0,4.0,Low,Low,Very Bad,1,16.0,14.0,16.0,Yes
1,School C,Female,14.0,Suburban,4.0,Apart,Higher,NaN,Service,Education,Course,Mother,7.0,4.0,0,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Very Bad,3.0,1.0,Very High,NaN,Good,3,15.0,16.0,15.0,No
2,School C,Female,18.0,Rural,5.0,Apart,NaN,Primary,Business,Unemployed,Course,Father,NaN,1.0,0,No,No,Yes,Yes,No,No,No,Yes,Bad,2.0,5.0,Very High,Medium,Excellent,0,17.0,13.0,8.0,Yes
3,School B,Other,18.0,Suburban,7.0,Apart,Secondary,Secondary,Business,Technology,Home,Other,41.0,4.0,1,No,No,Yes,No,Yes,Yes,No,Yes,Good,5.0,NaN,Low,NaN,Good,6,NaN,16.0,16.0,No
4,School A,Other,16.0,Suburban,8.0,Apart,Higher,NaN,Unemployed,Education,Other,Father,39.0,7.0,2,Yes,No,Yes,No,Yes,No,No,No,Very Bad,4.0,4.0,NaN,Low,Bad,2,16.0,13.0,15.0,No


In [11]:
print("This dataset has {} rows and {} columns".format(df.shape[0],df.shape[1]))

This dataset has 20000 rows and 34 columns


In [24]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   School                       20000 non-null  object 
 1   Gender                       20000 non-null  object 
 2   Age                          18977 non-null  float64
 3   Address                      20000 non-null  object 
 4   Family_Size                  19008 non-null  float64
 5   Parental_Status              20000 non-null  object 
 6   Mother_Education             14876 non-null  object 
 7   Father_Education             14986 non-null  object 
 8   Mother_Job                   20000 non-null  object 
 9   Father_Job                   20000 non-null  object 
 10  Reason_for_Choosing_School   20000 non-null  object 
 11  Guardian                     20000 non-null  object 
 12  Travel_Time                  18990 non-null  float64
 13  Study_Time      

In [44]:
print("There are {} columns wiht missing data.".format(len((df.isnull().mean() * 100)[df.isnull().mean() * 100 > 0])))

There are 13 columns wiht missing data.


In [45]:
(df.isnull().mean() * 100)[df.isnull().mean() * 100 > 0]


Age                             5.115
Family_Size                     4.960
Mother_Education               25.620
Father_Education               25.070
Travel_Time                     5.050
Study_Time                      5.095
Free_Time                       5.230
Going_Out                       4.990
Weekend_Alcohol_Consumption    19.885
Weekday_Alcohol_Consumption    20.155
Grade_1                         5.050
Grade_2                         5.075
Final_Grade                     5.030
dtype: float64

#### Isolating categorical features

In [47]:
df.select_dtypes(include='object').columns

Index(['School', 'Gender', 'Address', 'Parental_Status', 'Mother_Education',
       'Father_Education', 'Mother_Job', 'Father_Job',
       'Reason_for_Choosing_School', 'Guardian', 'School_Support',
       'Family_Support', 'Extra_Paid_Class', 'Extra_Curricular_Activities',
       'Attended_Nursery', 'Wants_Higher_Education', 'Internet_Access',
       'In_Relationship', 'Family_Relationship', 'Weekend_Alcohol_Consumption',
       'Weekday_Alcohol_Consumption', 'Health_Status', 'Dropped_Out'],
      dtype='object')

In [57]:
print("At a first glance, there are {} categorical features.".format(len(df.select_dtypes(include='object').columns)))

At a first glance, there are 23 categorical features.


#### Isolating quantitative features

In [58]:
df.select_dtypes(exclude='object').columns

Index(['Age', 'Family_Size', 'Travel_Time', 'Study_Time', 'Number_of_Failures',
       'Free_Time', 'Going_Out', 'Number_of_Absences', 'Grade_1', 'Grade_2',
       'Final_Grade'],
      dtype='object')

In [60]:
print("At a first glance, there are {} categorical features.".format(len(df.select_dtypes(exclude='object').columns)))

At a first glance, there are 11 categorical features.


#### However Going_Out should be a categorical feature since it shows a scale of values.

In [61]:
df['Going_Out'] = df['Going_Out'].astype('str')